In [1]:
# Ensure you have ffmpeg installed locally

import whisper    
# Initialize Whisper model
model = whisper.load_model("large-v2")

In [3]:
import os
import shutil
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

from pydub import AudioSegment

class AudioFileHandler(FileSystemEventHandler):
    def __init__(self, model, output_dir, processed_dir):
        self.model = model
        self.output_dir = output_dir
        self.processed_dir = processed_dir

    def on_created(self, event):
        if not event.is_directory:
            self.process_file(event.src_path)

    def process_file(self, file_path):
        try:
            # Check if the file is an audio file
            if file_path.lower().endswith(('.wav', '.mp3', '.m4a')):
                print(f"Processing file: {file_path}")

                # Convert audio file to WAV if necessary
                if not file_path.lower().endswith('.wav'):
                    audio = AudioSegment.from_file(file_path)
                    output_audio_file = file_path[:-3] + "wav"
                    audio.export(output_audio_file, format="wav")
                    convert = True
                else:
                    output_audio_file = file_path

                # Transcribe audio using Whisper
                result = self.model.transcribe(output_audio_file)

                # Extract transcription text
                transcription_text = result['text']

                # Save transcription to a file
                transcript_file = os.path.join(self.output_dir, os.path.basename(file_path) + '.txt')
                with open(transcript_file, "w") as f:
                    f.write(transcription_text)

                print(f"Transcription saved to: {transcript_file}")

                # Move the processed audio file to another folder
                shutil.move(file_path, os.path.join(self.processed_dir, os.path.basename(file_path)))

                # If conversion occured, delete the converted file 
                if convert:
                    os.remove(output_audio_file)

                print(f"Moved {file_path} to {self.processed_dir}")

        except Exception as e:
            print(f"Error processing file {file_path}: {e}")

if __name__ == "__main__":
    watch_dir = "../audio_files_drop"  # Directory to watch for new files
    output_dir = "../transcripts"  # Directory to save transcripts
    processed_dir = "../processed_audios"  # Directory to move processed audio files

    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(processed_dir, exist_ok=True)

    # Initialize Whisper model
    model = whisper.load_model("large-v2")

    event_handler = AudioFileHandler(model, output_dir, processed_dir)
    observer = Observer()
    observer.schedule(event_handler, path=watch_dir, recursive=False)
    observer.start()

    print(f"Watching directory: {watch_dir}")

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

Watching directory: ../audio_files_drop
